In [19]:
import gym
import pandas as pd
import numpy as np

In [5]:
env = gym.make('Taxi-v3').env

In [7]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [9]:
env.reset()
env.render()

print('Action Space {}'.format(env.action_space))
print('State Space {}'.format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


In [10]:
state = env.encode(3, 1, 2, 0)
print("State:", state)

env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [30]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [16]:
env.s = 328

epochs = 0 
penalties, reward = 0, 0 

frames = []

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    
    if reward == -10:
        penalties += 1
        
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state, 
        'action': action,
        'reward': reward})
    
    epochs += 1
    
print('Timesteps taken: {}'.format(epochs))
print('Penalties incurred {}'.format(penalties))

Timesteps taken: 340
Penalties incurred 106


In [17]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i+1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(0.01)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 340
State: 0
Action: 5
Reward: 20


In [55]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [69]:
np.argmax(q_table[200])

0

In [92]:
%%time

import random
from IPython.display import clear_output

alpha = 0.1
gamma = 0.6
epsilon = 0.1

all_epochs = []
all_penalties = []

for i in range(1, 2):
    state = env.reset()
    
    epochs, penalties, reward = 0, 0, 0
    done = False
    
    while not done:
        
        if random.uniform(0,1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state])
            
        next_state, reward, done, info = env.step(action)
        
        old_value = q_table[state, action]
        next_max = np.argmax(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value
        
        if reward == -10:
            penalties += 1
            
        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")
        
print(f"Training Complete.\n")

KeyboardInterrupt: 

In [20]:
q_table = pd.read_csv('Taxi_QTable.csv').to_numpy()

In [30]:
from IPython.display import clear_output
from time import sleep

def print_frame(frame):

    clear_output(wait=True)
    print(frame['frame'])
    print(f"Timestep: {epochs}")
    print(f"State: {state}")
    print(f"Action: {frame['action']}")
    print(f"Reward: {frame['reward']}")
    sleep(0.1)
        

In [32]:
epochs = 0 
penalties, reward = 0, 0 

env.reset()
done = False

for _ in range(5):
    
    action = np.argmax(q_table[state])
    state, reward, done, info = env.step(action)
    
    if reward == -10:
        penalties += 1
    
    epochs += 1
    
    frame = {'frame': env.render(mode='ansi'),
             'timestep': epochs,
             'state': state, 
             'action': action,
             'reward': reward}
    
    print_frame(frame)
    
print('Timesteps taken: {}'.format(epochs))
print('Penalties incurred {}'.format(penalties))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)

Timestep: 5
State: 474
Action: 0
Reward: -1
Timesteps taken: 5
Penalties incurred 0
